# Complementaria Repaso Quiz 3 - Martes 12:30

In [7]:
import numpy as np
import pandas as pd
from jmarkov.mdp.dtmdp import dtmdp


In [8]:
# Matrices de transición (orden de estados: O, D, C)

P_S = np.array([
    [0.85, 0.15, 0.00],
    [0.60, 0.35, 0.05],
    [0.40, 0.40, 0.20]
])

P_M = np.array([
    [0.70, 0.25, 0.05],
    [0.25, 0.55, 0.20],
    [0.10, 0.40, 0.50]
])

P_N = np.array([
    [0.60, 0.30, 0.10],
    [0.10, 0.50, 0.40],
    [0.00, 0.20, 0.80]
])

# Ingresos por estado (USD por semana)
ingresos = {
    'O': 300,
    'D': 200,
    'C': 80
}

# Costos por acción (USD por semana)
costos = {
    'S': -120,   # Servicio preventivo
    'M': -60,    # Mantenimiento parcial
    'N': 0       # No intervenir
}


## Parte A: Implementación del MDP

In [9]:
# Factor de descuento

beta = 0.8


# Espacio de estados y acciones

estados = np.array(["O","D","C"])

acciones = np.array(["S","M","N"])


# S: Servicio preventivo, M: Mantenimiento parcial, N: No intervenir


# Diccionario de matrices de transición por acción

matrices = {}

for a in acciones:
    if a == "S":
        matrices["S"]=P_S
    elif a == "M":
        matrices["M"]=P_M
    else:
        matrices["N"]=P_N

#print(matrices)


# Matriz de retornos R(s,a) = ingreso(s) + costo(a)

retornos = np.zeros((len(estados),len(estados)))

for i, s in enumerate(estados):
    for j, a in enumerate(acciones):
        retornos[i,j]=ingresos[s]+costos[a]

print(retornos)
# filas: estados [O, D, C], columnas: acciones [S, M, N]

beta = 0.8

mdp = dtmdp(estados, acciones, matrices, retornos, beta)


[[180. 240. 300.]
 [ 80. 140. 200.]
 [-40.  20.  80.]]


In [12]:


value_functions, optimal_policy = mdp.solve(0,method="value_iteration")

print(value_functions)

# Politica optima

print(optimal_policy)

# valor esperado bajo pi*

mdp.expected_policy_value(value_functions, optimal_policy)

# matriz de transicion

matriz_optima = mdp.policy_transition_matrix(optimal_policy)



matrizPO_df = pd.DataFrame(matriz_optima, index=estados, columns=estados)
matrizPO_df

[1089.16967505  876.17328516  701.08303245]
{np.str_('O'): np.str_('N'), np.str_('D'): np.str_('S'), np.str_('C'): np.str_('S')}


,O,D,C
O,0.6,0.30,0.10
D,0.6,0.35,0.05
C,0.4,0.40,0.20


## Parte B: Simulación de Monte Carlo

In [ ]:
escenarios = 1500        # (1000, 1200, 1500...)
horizon = 52        # (52 semanas, 20 meses, 24 horas, etc.)
estado_inicial = 'O'  # depende del problema

retornos_df = pd.DataFrame(retornos, index=estados, columns=acciones)

np.random.seed(0)

# Matriz principal: retorno/costo/peces/etc. por (tiempo, escenario)
Y = np.zeros((horizon, escenarios))

# Indicadores por estado (solo si preguntan tiempo en un estado)
indicadores_estados = {s: np.zeros((horizon, escenarios)) for s in estados}

# Indicador de evento al menos una vez (solo si preguntan probabilidad de visita)
indicador_evento = np.zeros(escenarios)

for j in range(escenarios):
    estado = estado_inicial
    visito_evento = 0

    for t in range(horizon):
        # acción óptima
        accion = str(optimal_policy[estado])

        # retorno/costo instantáneo
        Y[t, j] = retornos_df.loc[estado, accion]

        # marcar si estoy en un estado
        # (activar solo si el ejercicio menciona un estado específico)
        if estado == "C":
            indicadores_estados["ESTADO_OBJETIVO"][t, j] = 1

        # marcar evento temprano
        # (activar solo si el ejercicio menciona "primeros K períodos")
        #if t < K and estado == ESTADO_EVENTO:
        #    visito_evento = 1

        # transición por política óptima
        estado = np.random.choice(estados, p=matrizPO_df.loc[estado, :].values)

    indicador_evento[j] = visito_evento

In [20]:
# ¿Cu´al es el valor esperado de ganancias para la ´ultima semana?
respuesta = Y[-1, :].mean()
print(respuesta)

# porcentaje en critico
respuesta = indicadores_estados["C"].mean() * 100
print(respuesta)

193.33333333333334
9.007692307692308
